### With predicted rental prices, we are now interested in the growth rates for each SA2

In [30]:
import pandas as pd
import glob
import os

#path = r'../data/curated/2023_2027_rental_prediction/' # use your path
path = r'../data/curated/random_forest_pred/'
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

merged_df = pd.concat(li, axis=0, ignore_index=True)
predicted = merged_df.groupby(['year', 'sa2_2021'], as_index=False).agg({'predicted_price': 'mean'})
predicted

,year,sa2_2021,predicted_price
0,2023,201011001,749.428642
1,2023,201011002,681.888264
2,2023,201011005,699.825114
3,2023,201011006,822.960284
4,2023,201011007,776.648767
...,...,...,...
2490,2027,217031476,551.693307
2491,2027,217041477,563.539169
2492,2027,217041478,565.535547
2493,2027,217041479,559.410434


In [31]:
rental_2022 = pd.read_csv('../data/curated/merged_dataset/2022_merged_data.csv')
rental_2022.dropna(inplace=True)
rental_2022 = rental_2022.groupby(['year', 'sa2_2021'], as_index=False).agg({'weekly_rent': 'mean'})
rental_2022.rename({'weekly_rent': 'predicted_price'}, axis=1, inplace=True)
rental_2022
#rental_2022[rental_2022['sa2_2021']==206041127]
#set(rental_2022['sa2_2021'].unique()) - set(predicted['sa2_2021'].unique())

,year,sa2_2021,predicted_price
0,2022,201011001,437.468013
1,2022,201011002,421.270053
2,2022,201011005,402.559524
3,2022,201011006,440.000000
4,2022,201011007,530.625000
...,...,...,...
494,2022,217031476,518.750000
495,2022,217041477,414.069767
496,2022,217041478,771.301370
497,2022,217041479,437.025641


In [32]:
growth_df = pd.concat([rental_2022, predicted], axis=0).reset_index(drop=True)
#df[df['sa2_2021']==201011001]
growth_df

,year,sa2_2021,predicted_price
0,2022,201011001,437.468013
1,2022,201011002,421.270053
2,2022,201011005,402.559524
3,2022,201011006,440.000000
4,2022,201011007,530.625000
...,...,...,...
2989,2027,217031476,551.693307
2990,2027,217041477,563.539169
2991,2027,217041478,565.535547
2992,2027,217041479,559.410434


In [33]:
growth_dict = dict()
for sa2 in growth_df['sa2_2021'].unique():
    growth_dict[sa2] = []
    subset = growth_df[growth_df['sa2_2021']==sa2]
    for i in range(len(subset['predicted_price'])-1):
        # (current year price - past year price) / past year price
        growth_rate = (subset.iloc[i+1]['predicted_price'] - subset.iloc[i]['predicted_price'])/subset.iloc[i]['predicted_price']
        growth_dict[sa2].append(growth_rate)


In [34]:
growth_dict

{201011001: [0.7131050009688057, 0.0, 0.0, 0.0, 0.0],
 201011002: [0.6186487946262712, 0.0, 0.0, 0.0, 0.0],
 201011005: [0.7384388448386295, 0.0, 0.0, 0.0, 0.0],
 201011006: [0.8703642809443742, 0.0, 0.0, 0.0, 0.0],
 201011007: [0.4636490311949369, 0.0, 0.0, 0.0, 0.0],
 201011008: [1.0206144084154487, 0.0, 0.0, 0.0, 0.0],
 201011481: [0.9996722787909716, 0.0, 0.0, 0.0, 0.0],
 201011482: [0.9205986553407464, 0.0, 0.0, 0.0, 0.0],
 201011483: [0.9874621417650209, 0.0, 0.0, 0.0, 0.0],
 201011484: [1.2510782345733724, 0.0, 0.0, 0.0, 0.0],
 201021009: [0.5788978525614221, 0.0, 0.0, 0.0, 0.0],
 201021010: [0.6843201889583528, 0.0, 0.0, 0.0, 0.0],
 201021011: [0.3112895168145563, 0.0, 0.0, 0.0, 0.0],
 201021012: [0.40683410954904575, 0.0, 0.0, 0.0, 0.0],
 201031013: [1.006835356748298, 0.0, 0.0, 0.0, 0.0],
 201031014: [0.8840762928240992, 0.0, 0.0, 0.0, 0.0],
 201031015: [0.6459980010635492, 0.0, 0.0, 0.0, 0.0],
 201031016: [0.9427132920051259, 0.0, 0.0, 0.0, 0.0],
 201031017: [0.9959507063798

In [35]:
import numpy as np
for key in growth_dict.keys():
    growth_dict[key] = np.mean(growth_dict[key])
growth_dict

{201011001: 0.14262100019376114,
 201011002: 0.12372975892525424,
 201011005: 0.1476877689677259,
 201011006: 0.17407285618887486,
 201011007: 0.09272980623898738,
 201011008: 0.20412288168308973,
 201011481: 0.19993445575819432,
 201011482: 0.18411973106814927,
 201011483: 0.19749242835300418,
 201011484: 0.25021564691467446,
 201021009: 0.11577957051228442,
 201021010: 0.13686403779167056,
 201021011: 0.06225790336291126,
 201021012: 0.08136682190980915,
 201031013: 0.2013670713496596,
 201031014: 0.17681525856481983,
 201031015: 0.12919960021270985,
 201031016: 0.1885426584010252,
 201031017: 0.19919014127597068,
 202011018: 0.14534717778993927,
 202011019: 0.1693785073134237,
 202011020: 0.12012007939291161,
 202011021: 0.1508350609240489,
 202011022: 0.13840708005510785,
 202011023: 0.10110103975011309,
 202011024: 0.11338694374229996,
 202011025: 0.13557257026225025,
 202021026: 0.11522920372052636,
 202021027: 0.12921987088111844,
 202021028: 0.13511703079204973,
 202021029: 0.1

In [36]:
# Sort dictionary so that the dictionary values are sorted as descneding order 
# The first subrub in the dictionary = the suburb which is predicted to have the highest average growth rate in the next 5 years 
sorted_dict = dict(sorted(growth_dict.items(), key=lambda item: item[1], reverse=True))
sorted_dict_list = list(sorted_dict)

sa2_list = list(sorted_dict.keys())
growth_list = list(sorted_dict.values())

top_ten_names = []

sa2_dict = pd.read_csv("../data/curated/sa2_vic_2021.csv")

i=0
while i < len(sa2_list):
    selected = sa2_list[i]
    suburb_name = sa2_dict[sa2_dict['SA2_CODE21']==selected]['SA2_NAME21'].to_list()[0]
    top_ten_names.append(suburb_name)
    i+=1

# Make a dataframe with the prediced results with columns of suburb name, its sa2 code and its growth rate
new_csv_name = "../data/curated/final_growing_rates_rf.csv" 

data = {'Suburb_Name': top_ten_names,
        'sa2_2021': sa2_list,
        'Growth_Rate': growth_list}

df = pd.DataFrame(data)
print(df)
df.to_csv(new_csv_name, index=False)

                 Suburb_Name   sa2_2021  Growth_Rate
0                     Loddon  202031033     0.372790
1           Ararat Surrounds  215011387     0.310035
2                    Morwell  205041096     0.259896
3         Sebastopol - Redan  201011484     0.250216
4               Yarriambiack  215011394     0.244375
..                       ...        ...          ...
494             Moyne - West  217041478    -0.053355
495          Bundoora - East  209011196    -0.054011
496                   Echuca  216011406    -0.059837
497                 Flinders  214021378    -0.065283
498  Donvale - Park Orchards  211021261    -0.076247

[499 rows x 3 columns]
